In [2]:
import os
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import plotly.graph_objects as go
from plotly.subplots import make_subplots


catalog_path = 'ges_catalog'
output_folder = 'data_statistics'
show_images = False
width_mm = 85
dpi = 300
width_in = width_mm / 25.4
height_in = width_in * 0.75


dataframes = []
for file_name in os.listdir(catalog_path):
    file_path = os.path.join(catalog_path, file_name)
    if file_name.endswith('.csv'):
        df = pd.read_csv(file_path)
        if not df.empty:
            dataframes.append(df)

catalog_df = pd.concat(dataframes, ignore_index=True)
catalog_df.columns = catalog_df.columns.str.strip().str.replace(' ', '_')

catalog_df = catalog_df[catalog_df['Depth'] > 0]

os.makedirs(output_folder, exist_ok=True)

# 1. Plot statistics for Depth
fig_depth = px.histogram(catalog_df, x="Depth", nbins=100, title="Histogram of Depth")

depth_mean = catalog_df['Depth'].mean()
depth_std = catalog_df['Depth'].std()
depth_median = catalog_df['Depth'].median()
depth_ci = stats.norm.interval(0.95, loc=depth_mean, scale=depth_std)

fig_depth.add_vline(x=depth_mean, line=dict(color="red", dash="dash"), annotation_text=f"Mean {depth_mean:.2f}", annotation_position="top left")
fig_depth.add_vline(x=depth_mean + depth_std, line=dict(color="green", dash="dash"), annotation_text="+1 Std Dev", annotation_position="top left")
fig_depth.add_vline(x=depth_mean - depth_std, line=dict(color="green", dash="dash"), annotation_text="-1 Std Dev", annotation_position="top left")

fig_depth.update_layout(
            width=800,
            height=600,
            title_x=0.5,
            margin=dict(l=40, r=40, b=40, t=40),
        )

if show_images:
    fig_depth.show()
else:
    fig_depth.write_image(os.path.join(output_folder,"depth_histogram.png"), width=int(width_in * dpi), height=int(height_in * dpi), scale=2)

print(f"Depth Mean: {depth_mean}")
print(f"Depth Std: {depth_std}")
print(f"Depth Median: {depth_median}")
print(f"Depth 95% CI: {depth_ci}")


# 2. Create a 1x1 subplot for scatter plot and legend
fig = make_subplots(
    rows=1, cols=1
)

scatter = go.Scatter(
    x=catalog_df['X'], 
    y=catalog_df['Y'],
    mode='markers',
    marker=dict(
        size = (catalog_df['MomMag'] - catalog_df['MomMag'].min() + 0.4)*5,
        color=catalog_df['Depth'],
        colorscale='Magma',
        colorbar=dict(title="Depth"),
        line=dict(width=0.5),
        opacity=0.7
    )
)

fig.add_trace(scatter)
fig.update_layout(
    title="2D Scatter Plot of X, Y Event Coordinates",
    width=800,
    height=600,
    title_x=0.5,
    margin=dict(l=40, r=40, b=40, t=40),
    xaxis_title="X",
    yaxis_title="Y"
)

if show_images:
    fig.show()
else:
    fig.write_image(os.path.join(output_folder,"x_y_scatter.png"), width=int(width_in * dpi), height=int(height_in * dpi), scale=2)


# 3. Plot statistics for MomMag
fig_mommag = px.histogram(catalog_df, x="MomMag", nbins=100, title="Histogram of Moment Magnitudes")

mommag_mean = catalog_df['MomMag'].mean()
mommag_std = catalog_df['MomMag'].std()
mommag_median = catalog_df['MomMag'].median()
mommag_ci = stats.norm.interval(0.95, loc=mommag_mean, scale=mommag_std)

fig_mommag.add_vline(x=mommag_mean, line=dict(color="red", dash="dash"), annotation_text=f"Mean {mommag_mean:.2f}", annotation_position="top left")
fig_mommag.add_vline(x=mommag_mean + mommag_std, line=dict(color="green", dash="dash"), annotation_text="+1 Std Dev", annotation_position="top left")
fig_mommag.add_vline(x=mommag_mean - mommag_std, line=dict(color="green", dash="dash"), annotation_text="-1 Std Dev", annotation_position="top left")
fig_mommag.update_layout(
    showlegend=True,
    width=800,
    height=600,
    title_x=0.5,
    margin=dict(l=40, r=40, b=40, t=40),
)

if show_images:
    fig_mommag.show()
else:
    fig_mommag.write_image(os.path.join(output_folder,"magnitude_histogram.png"), width=int(width_in * dpi), height=int(height_in * dpi), scale=2)

print(f"MomMag Mean: {mommag_mean}")
print(f"MomMag Std: {mommag_std}")
print(f"MomMag Median: {mommag_median}")
print(f"MomMag 95% CI: {mommag_ci}")

Depth Mean: 7716.648487853247
Depth Std: 435.6767813716188
Depth Median: 7750.0
Depth 95% CI: (6862.737687464543, 8570.559288241951)
MomMag Mean: -0.4573326722855727
MomMag Std: 0.4651161794704701
MomMag Median: -0.44
MomMag 95% CI: (-1.368943632674562, 0.4542782881034168)
